In [1]:
!pip install -q transformers trl datasets wandb torch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade accelerate==1.6.0
!pip install --upgrade transformers==4.51.3


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from trl import RewardTrainer, RewardConfig
from datasets import load_dataset
import warnings

warnings.filterwarnings('ignore')

2025-06-09 19:40:33.397963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749487233.483570 1196640 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749487233.508674 1196640 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749487233.707280 1196640 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749487233.707293 1196640 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749487233.707294 1196640 computation_placer.cc:177] computation placer alr

In [4]:
wandb_logging = True
pretrained_model = "HuggingFaceTB/SmolLM2-135M-Instruct"  
reward_learning_rate = 5e-5
reward_training_cycles = 1
max_sequence_len = 512  
training_batch_size = 8  
validation_batch_size = 8  
reward_model_save_path = os.path.join(os.getcwd(), "trained_reward_model")  

reinforce_batch_dim = 4  
reinforce_learning_rate = 5e-5
reinforce_epoch_count = 1
max_generated_tokens = 512
reinforce_output_path = os.path.join(os.getcwd(), "reinforce_baseline")  

os.makedirs(reward_model_save_path, exist_ok=True)  
os.makedirs(reinforce_output_path, exist_ok=True)

In [5]:
def get_tokenizer(tokenizer_path: str):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, padding_side="left")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

def extract_text(conversation):
    if isinstance(conversation, str):
        return conversation
    if isinstance(conversation, list):
        return "\n".join(
            [f"{turn['role'].capitalize()}: {turn['content']}"
             if isinstance(turn, dict)
             else str(turn)
             for turn in conversation]
        )
    return str(conversation)


def edit_reward_dataset(examples, tokenizer):
    # Обработка батча примеров
    chosen = tokenizer(
        examples["chosen_text"],
        truncation=True,
        padding="max_length",
        max_length=max_sequence_len,
        return_tensors="pt",
    )
    
    rejected = tokenizer(
        examples["rejected_text"],
        truncation=True,
        padding="max_length",
        max_length=max_sequence_len,
        return_tensors="pt",
    )
    
    return {
        "input_ids_chosen": chosen["input_ids"],
        "attention_mask_chosen": chosen["attention_mask"],
        "input_ids_rejected": rejected["input_ids"],
        "attention_mask_rejected": rejected["attention_mask"],
    }

def edit_reinforce_dataset(batch, tokenizer):
    prompts = []
    for conversation in batch["chosen"]:
        if isinstance(conversation, list):
            prompt = "\n".join(
                [f"{turn['role'].capitalize()}: {turn['content']}" for turn in conversation[:-1]]
            ) + "\nAssistant: "
        else:
            prompt = str(conversation) + "\nAssistant: "
        prompts.append(prompt)

    tokens = tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=max_sequence_len,
    )

    return {
        k: v for k, v in tokens.items()
    }

def splitting_dataset(test_size=0.2, train_subset_sizes=None, val_subset_size=None):
    dataset = load_dataset("juyoungml/HelpSteer2-binarized")

    if "train" in dataset:
        train_dataset = dataset["train"]
        split_dataset = train_dataset.train_test_split(test_size=test_size, seed=42)
        dataset["train"] = split_dataset["train"]
        dataset["validation"] = split_dataset["test"]

    if train_subset_sizes is not None:
        dataset["train"] = dataset["train"].select(range(train_subset_sizes))

    if val_subset_size is not None and "validation" in dataset:
        dataset["validation"] = dataset["validation"].select(range(val_subset_size))

    return dataset

In [7]:
import torch
from trl import RewardTrainer


class train_reward_model2(RewardTrainer):
    def compute_loss(
        self,
        model,
        inputs: dict[str, torch.Tensor],
        return_outputs: bool = False,
        num_items_in_batch=None,
    ) -> torch.Tensor | tuple[torch.Tensor, dict[str, torch.Tensor]]:
        logits_chosen = model(
            input_ids=inputs["input_ids_chosen"],
            attention_mask=inputs["attention_mask_chosen"],
            return_dict=True,
        )["logits"]
        logits_rejected = model(
            input_ids=inputs["input_ids_rejected"],
            attention_mask=inputs["attention_mask_rejected"],
            return_dict=True,
        )["logits"]

        probs_chosen = torch.softmax(logits_chosen, dim=-1)
        probs_rejected = torch.softmax(logits_rejected, dim=-1)

        batch_size, num_classes = probs_chosen.shape
        mask = torch.tril(
            torch.ones(num_classes, num_classes, device=probs_chosen.device),
            diagonal=-1,
        )
        prod = probs_chosen.unsqueeze(2) * probs_rejected.unsqueeze(
            1
        )  # prod[., i, j] = chosen[., i] * rejected[., j]
        total_probability = (prod * mask).sum(dim=(1, 2))

        loss = -torch.log(total_probability.clamp(min=1e-8)).mean()

        if self.args.center_rewards_coefficient is not None:
            pass

        if return_outputs:
            return loss, {
                "probs_chosen": probs_chosen,
                "probs_rejected": probs_rejected,
            }
        return loss

    def prediction_step(
        self,
        model,
        inputs: dict[str, torch.Tensor],
        prediction_loss_only: bool,
        ignore_keys: list[str] | None = None,
    ) -> tuple[torch.Tensor | None, torch.Tensor | None, torch.Tensor | None]:
        inputs = self._prepare_inputs(inputs)
        if ignore_keys is None:
            if hasattr(self.model, "config"):
                ignore_keys = getattr(
                    self.model.config, "keys_to_ignore_at_inference", []
                )
            else:
                ignore_keys = []

        with torch.no_grad():
            loss, logits_dict = self.compute_loss(model, inputs, return_outputs=True)

        if prediction_loss_only:
            return (loss, None, None)

        loss = loss.detach()

        probs_chosen = logits_dict["probs_chosen"]
        probs_rejected = logits_dict["probs_rejected"]
        ratings = torch.arange(
            1,
            probs_chosen.size(-1) + 1,
            device=probs_chosen.device,
            dtype=probs_chosen.dtype,
        )  # [1, 2, ... num_labels]
        expected_chosen = (probs_chosen * ratings).sum(dim=-1)
        expected_rejected = (probs_rejected * ratings).sum(dim=-1)
        logits = torch.stack([expected_chosen, expected_rejected], dim=-1)

        labels = torch.ones(
            expected_chosen.size(0), device=expected_chosen.device, dtype=torch.int32
        )

        return loss, logits, labels

In [13]:
import sys
import os
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification
from trl import RewardConfig

if wandb_logging:
    import wandb

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    preds = (predictions[:, 0] > predictions[:, 1]).astype(np.int32)
    accuracy = (preds == labels).mean() if labels.size > 0 else 0.0
    return {"accuracy": accuracy}


def train_reward_model(output_model_dir):
    if wandb_logging:
        wandb.init(project="huggingface")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    tokenizer = get_tokenizer(pretrained_model)
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_model, num_labels=10
    ).to(device)

    for param in model.parameters():
        param.requires_grad = True

    dataset = splitting_dataset(test_size=0.2)

    dataset = dataset.map(
        lambda ex: edit_reward_dataset(ex, tokenizer),
        batched=True,
        batch_size=8,
        remove_columns=dataset["train"].column_names
    )

    train_dataset = dataset["train"]
    eval_dataset = dataset["validation"]

    second_training_epochs = 2
    second_dir = os.path.join(os.getcwd(), "trained_second_model")

    training_args = RewardConfig(
        output_dir=output_model_dir,
        per_device_train_batch_size=training_batch_size,
        per_device_eval_batch_size=validation_batch_size,
        num_train_epochs=second_training_epochs,
        learning_rate=reward_learning_rate,
        gradient_accumulation_steps=1,
        fp16=True,
        eval_steps=200,
        logging_steps=10,
        max_length=max_sequence_len,
        remove_unused_columns=True,
        report_to="none",
        gradient_checkpointing=True,
    )

    trainer = train_reward_model2(
        model=model,
        args=training_args,
        processing_class=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    init_metrics = trainer.evaluate()
    print("Initial eval metrics:", init_metrics)
    if wandb_logging:
        wandb.log(init_metrics)

    trainer.train()

    trainer.save_model(output_model_dir)
    print("Reward model saved to:", output_model_dir)

    post_metrics = trainer.evaluate()
    print("Post-training eval metrics:", post_metrics)
    if wandb_logging:
        wandb.log(post_metrics)
        wandb.finish()

output_model_dir = "second_dir"
train_reward_model(output_model_dir=output_model_dir)

eval_accuracy,▁
eval_loss,▁
eval_model_preparation_time,▁
eval_runtime,▁
eval_samples_per_second,▁
eval_steps_per_second,▁
eval_accuracy,0.49135
eval_loss,1.47742
eval_model_preparation_time,0.0027
eval_runtime,8.4744
eval_samples_per_second,170.513


Device: cuda


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1445 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Step,Training Loss
10,1.121900
20,0.922500
30,0.844800
40,0.801000
50,0.859600
60,0.839300
70,0.860600
80,0.760000
90,0.790000
100,0.857000


Reward model saved to: second_dir


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ You make a valid point that AI systems can    │ While an AI system can exhibit some of the   │ [6.1415, 2.9997] │
│ exhibit some characteristics that are similar │ characteristics of life, it is not truly     │                  │
│ to those of living organisms. However, it is  │ alive as it lacks the ability to             │                  │
│ important to note that AI systems are still   │ self-replicate and evolve through natural    │                  │
│ artificial constructs, and they do not        │ selection.                                   │                  │
│ possess the full range of characteristics     │                                              │                  │
│ that are typically associated with life.      │                                              │                  │
│                                               │                                              │                  │
│                                               │                                              │                  │
│  While AI systems can learn and adapt to      │                                              │                  │
│ their environment, they do not have the       │                                              │                  │
│ ability to self-replicate or to maintain      │                                              │                  │
│ homeostasis in the same way that living       │                                              │                  │
│ organisms do. They also do not have the       │                                              │                  │
│ capacity for consciousness or self-awareness, │                                              │                  │
│ which are key features of life as we know it. │                                              │                  │
│                                               │                                              │                  │
│                                               │                                              │                  │
│  Furthermore, the process by which AI systems │                                              │                  │
│ improve themselves is based on algorithms and │                                              │                  │
│ mathematical models, rather than on the       │                                              │                  │
│ natural processes that occur in living        │                                              │                  │
│ organisms. While AI systems can use           │                                              │                  │
│ reinforcement learning to improve their       │                                              │                  │
│ performance, this process is still based on   │                                              │                  │
│ mathematical calculations and data analysis,  │                                              │                  │
│ rather than on the biological processes that  │                                              │                  │
│ occur in living systems.                      │                                              │                  │
│                                               │                                              │                  │
│                                               │                                              │                  │
│  In summary, while AI systems can exhibit     │                                              │                  │
│ some characteristics that are similar to      │       

Post-training eval metrics: {'eval_loss': 0.8040561079978943, 'eval_model_preparation_time': 0.0026, 'eval_accuracy': 0.6325259515570935, 'eval_runtime': 8.6692, 'eval_samples_per_second': 166.681, 'eval_steps_per_second': 20.878, 'epoch': 2.0}


epoch,▁
eval_accuracy,▁█
eval_loss,█▁
eval_model_preparation_time,▁▁
eval_runtime,▁█
eval_samples_per_second,█▁
eval_steps_per_second,█▁
epoch,2
eval_accuracy,0.63253
eval_loss,0.80406
eval_model_preparation_time,0.0026


In [25]:
def edit_reinforce_dataset2(example, tokenizer):
    encoding = tokenizer(
        example["chosen"],
        padding="max_length",
        truncation=True,
        max_length=max_sequence_len,
    )
    return {
        "input_ids": encoding["input_ids"],
        "attention_mask": encoding["attention_mask"],
    }

In [27]:
import sys
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
)

if wandb_logging:
    import wandb

def train_reinforce(reward_model_path, output_model_dir):
    if wandb_logging:
        wandb.init(
            project="huggingface",
            config={
                "batch_size": reinforce_batch_dim,
                "learning_rate": reinforce_learning_rate,
            },
        )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = get_tokenizer(pretrained_model)
    model = AutoModelForCausalLM.from_pretrained(pretrained_model).to(device)

    reward_tokenizer = get_tokenizer(pretrained_model)
    reward_model = AutoModelForSequenceClassification.from_pretrained(
        reward_model_path
    ).to(device)
    reward_model.eval()

    dataset = splitting_dataset(
        test_size=0.2, train_subset_sizes=1000, val_subset_size=300
    )
    train_dataset = dataset["train"].map(
        lambda ex: edit_reinforce_dataset2(ex, tokenizer)
    )
    val_dataset = dataset["validation"].map(
        lambda ex: edit_reinforce_dataset2(ex, tokenizer)
    )
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

    train_loader = DataLoader(
        train_dataset, batch_size=reinforce_batch_dim, shuffle=True
    )
    val_loader = DataLoader(val_dataset, batch_size=reinforce_batch_dim)

    optimizer = torch.optim.AdamW(model.parameters(), lr=reinforce_learning_rate)

    def compute_rewards(generated_sequences, input_ids):
        texts = reward_tokenizer.batch_decode(
            generated_sequences, skip_special_tokens=True
        )

        inputs = reward_tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=max_sequence_len,
            return_tensors="pt",
        ).to(device)

        with torch.no_grad():
            logits = reward_model(**inputs).logits  # shape: [batch_size, num_labels=10]
            probabilities = torch.softmax(logits, dim=-1)
            ratings = torch.arange(
                1, 11, dtype=probabilities.dtype, device=probabilities.device
            )
            # ratings = ratings * -1
            expected_rewards = (probabilities * ratings).sum(dim=-1)

        return expected_rewards

    def evaluate(model, dataloader):
        model.eval()
        all_rewards = []
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Evaluation"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                generated = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=max_generated_tokens,
                    pad_token_id=tokenizer.pad_token_id,
                )
                rewards = compute_rewards(generated, input_ids)
                all_rewards.extend(rewards.cpu().numpy())
        return np.mean(all_rewards), all_rewards

    mean_reward_before, all_rewards_before = evaluate(model, val_loader)
    print(f"Initial mean reward: {mean_reward_before:.4f}")
    if wandb_logging:
        wandb.log(
            {
                "eval/mean_reward_before": mean_reward_before,
                "eval/all_rewards_before": all_rewards_before,
            }
        )

    total_rewards = 0.0
    step_count = 0
    baseline = 0.0

    for epoch in range(reinforce_epoch_count):
        model.train()

        for batch_idx, batch in enumerate(tqdm(train_loader, desc="Training")):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_generated_tokens,
                pad_token_id=tokenizer.pad_token_id,
                return_dict_in_generate=True,
                output_scores=True,
            )

            full_sequences = outputs.sequences  # input + generated
            generated_tokens = full_sequences[:, input_ids.shape[1] :]

            rewards = compute_rewards(full_sequences, input_ids)

            gen_attention_mask = torch.ones(
                generated_tokens.shape, dtype=torch.long, device=device
            )
            full_attention_mask = torch.cat([attention_mask, gen_attention_mask], dim=1)

            logits = model(full_sequences, attention_mask=full_attention_mask).logits
            log_probs = torch.log_softmax(logits[:, :-1], dim=-1)

            selected_log_probs = log_probs.gather(
                -1, generated_tokens.unsqueeze(-1)
            ).squeeze(-1)
            aggregated_log_probs = selected_log_probs.sum(dim=1)

            total_rewards += rewards.sum().item()
            step_count += rewards.shape[0]
            baseline = total_rewards / step_count
            baseline_tensor = torch.tensor(baseline, device=device)
            advantages = rewards - baseline_tensor

            loss = -(advantages * aggregated_log_probs).mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if wandb_logging:
                wandb.log(
                    {
                        "train/loss": loss.item(),
                        "train/avg_reward": rewards.mean().item(),
                        "train/avg_advantage": advantages.mean().item(),
                        "train/baseline": baseline,
                        "train/step": batch_idx,
                    }
                )

            if batch_idx % 10 == 0:
                print(
                    f"Batch {batch_idx}: loss: {loss.item():.4f}, avg_reward: {rewards.mean().item():.4f}"
                )

    mean_reward_after, all_rewards_after = evaluate(model, val_loader)
    print(f"Final mean reward: {mean_reward_after:.4f}")
    if wandb_logging:
        wandb.log(
            {
                "eval/mean_reward_after": mean_reward_after,
                "eval/all_rewards_after": all_rewards_after,
            }
        )
        wandb.finish()

    os.makedirs(output_model_dir, exist_ok=True)
    model.save_pretrained(output_model_dir)
    print("Model saved to:", output_model_dir)

reward_model_path = "second_dir"
output_model_dir = "reinforce_alignment"
train_reinforce(reward_model_path=reward_model_path, output_model_dir=output_model_dir)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Evaluation: 100%|███████████████████████████████████████████████████████████████████████| 75/75 [07:35<00:00,  6.08s/it]


Initial mean reward: 5.5229


Training:   0%|▎                                                                        | 1/250 [00:00<03:31,  1.18it/s]

Batch 0: loss: 208.9564, avg_reward: 6.6012


...

Batch 240: loss: 20.7402, avg_reward: 7.2317


Evaluation: 100%|███████████████████████████████████████████████████████████████████████| 75/75 [06:01<00:00,  4.82s/it]

Final mean reward: 5.6536


eval/mean_reward_after,▁
eval/mean_reward_before,▁
train/avg_advantage,▄█▇▇▇▆▄▆▇▅▇▇▄▅▅▅█▄▇▅▇▅▆▃▅▂▆▅▅▃▅▅▆▄▁▄▇█▅█
train/avg_reward,▆▇▇▆▆▇▅▆██▁▄▆▃▄▅▆▆▆▄▆▇▅▇▃▆▅▇▆▄▅▇▆▄▅▄▂▅▃▂
train/baseline,▇▁█▇▇███████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
train/loss,▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▅▅▁▅█▅▁▅▅▅▄▅▅▅▄▅
train/step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
eval/mean_reward_after,5.65364
eval/mean_reward_before,5.52289
train/avg_advantage,-0.62796
train/avg_reward,5.95549


Model saved to: reinforce_alignment
